In [ ]:
import pickle as pkl
import glob
import numpy as np
import cv2
from img_load import load_bimg
from NPSmethods_Single import *
import os

In [ ]:
psat=277

#Settings of imaging system
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

#Configurations for NPSMethods (here we only need do_Fit)
choices = {
    "if_Save"                     : True,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : True,
    "plot_NoisePowSpec_LineCut"   : True,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

#Directory containing image folders
path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image'

#Directory where ML data will output to
resDir = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\results_single'

#Directory 
sinDir = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image_single'

trapRegion = (slice(100, 200), slice(100, 200))
    # the region where the atoms located, [(xmin, xmax), (ymin, ymax)] (pixel)

noiseRegion = (slice(0, 300), slice(0, 300))
    # the region chosen for noise analysis, [(xmin, xmax), (ymin, ymax)] (pixel)

numOfImgsInEachRun = 1      # number of images for each run of analysis

rampingParameter = 0

#Lists where OD images and parameters will be stored
rawimages = []
parameters = []
single_parameters = []

img_num = 0
#Sort through folders in path directory for raw images, extract OD image and parameters, then append to lists
for folder in glob.iglob(path + "\\*"):
    single_parameters_temp = []
    print(folder)
    for image_file in glob.iglob(folder + "\\rawimg_*"):
        print(image_file)
        img_num = img_num + 1
        if not os.path.exists(sinDir + "\\" + str(img_num)):
            os.mkdir(sinDir + "\\" + str(img_num))
        open1 = os.popen('copy "' + image_file + '" "' + sinDir + "\\" + str(img_num) + "\\rawimg_" + str(img_num) + '"')
        open2 = os.popen('copy "' + folder + '\\parameters.txt" "' + sinDir + "\\" + str(img_num) + '\\parameters.txt"')
        open1.close()
        open2.close()
        folder2 = sinDir + "\\" + str(img_num)
        print(folder2)
        result=load_bimg(folder2 + "\\", raw=True)
        print(1)
        odimg=-np.log(result['raw1']/result['raw2'])-(result['raw1']-result['raw2'])/psat
        _, _, _, _, popt, _ = doCalibration(folder2, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
        single_parameters_temp.append(popt)
    result=load_bimg(folder + "\\", raw=True)
    odimg=-np.log(result['raw1']/result['raw2'])-(result['raw1']-result['raw2'])/psat
    _, _, _, _, popt, _ = doCalibration(folder, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    print(popt)
    for i in range(len(single_parameters_temp)):
        single_parameters.append(single_parameters_temp[i])
        parameters.append(popt)

In [ ]:
#Save single-OD images and parameters to pickle files
pkl.dump(single_parameters, open(resDir + "\\single_parameters.pkl", "wb"))
pkl.dump(parameters, open(resDir + "\\parameters.pkl", "wb"))